
# Quick Start
----

**dSpace makes it easy to build simple, declarative, and composable abstractions in smart spaces.**

Define a digi schema:
```yaml
group: digi.dev
version: v1
kind: Plug
control:
  power: string
``` 

Program the digi's driver:
```python
from digi import on

# tuya device library
import pytuya
plug = pytuya.Plug("DEVICE_ID")

@on.control("power")
def h(power):
    plug.set(power["intent"])
```

Build and run the digi:
```bash
dq build plug; dq run plug plut-test
```

Update the intent of the digi in the model:
```yaml
apiVersion: digi.dev/v1
kind: Plug        
metadata:
  name: plug-test
spec:
  control:        
    power:
      # 'I want the plug switched off'
      intent: "off"
```

And apply the it via kubectl:
`kubectl apply -f MODEL_YAML`



## Concepts
----

**Digi:** The basic building block in dSpace is called _digi_. Each digi has a _model_ and a _driver_. A model consists of attribute-value pairs organized in a document (JSON) following a predefined _schema_. The model describes the desired and the current states of the digi; and the goal of the digi's driver is to take actions that reconcile the current states to the desired states. 

**Programming driver:** Each digi's driver can only access its own model (think of it as the digi's "world view"). Programming a digi is conceptually very simple - manipulating the digi's model/world view (essentially a JSON document) plus any other actions to affect the external world (e.g., send a signal to a physical plug, invoke a web API, send a text messages to your phone etc.). The dSpace's runtime will handle the rest, e.g., syncing states between digis' world views when they are composed.

**Build and run:** After defining the digi's schema and programing its driver, developers can build a _digi image_ and push it to a digi repository for distributiton. Users can then pull the digi image, run it on dSpace, and interact with the digi by updating its model, e.g., specifies its desired states. 

**Digivice:** In this tutorial, we will focus on a special type of digi called _digivice_ (e.g., the Plug in Quick Start). A digivice model has control attributes (e.g., `control.power` in Plug) where each control attribute has an intent field (tracking the desired states) and a status field (tracking the current states). 

Digivices can be composed via the _mount_ operator, forming digivice hierarchies.  Mounting a digi A to another digi B will allow B to change the intent fields and read the status fields of A (via updating the corresponding attribute replica of A in B's own model).

## How to use this notebook
----

This notebook contains a tutorial on building abstractions for a simple home smart space. You will run through each notebook cell serially. Make sure every cell before it has been run successfully before running the current cell. 

To run a notebook cell, you can click the `Run` button in the panel (or hit `shift + ENTER` as a shortcut). When a cell is run, its outputs will appear on the cell's output section. 

> Note: the notebook contains a few macros (e.g., `%elapsed_time`, `%%writefile`); you should be able to safely ignore them. Commands with a leading macro `!` are ones will be executed in shell.

# Home space
----

In this tutorial, we will learn how to implement a simple declarative space for home in dSpace. 

This example space includes `lamps`, `rooms`, `motion sensors`, and the `home`. We will compose these abstractions and define policies and automations. 

**We left a few lines of code and configurations to fill. They are marked by `YOUR CODE HERE`.** The tutorial should take about 15-30 min to walk through. 

## Setup and tools
----
We will be using two command line tools:

* **dq**: dSpace's CLI; used to build a digi-image (`dq build`), run a digi (`dq run`), and compose digivices (`dq mount CHILD PARENT`).
* **kubectl**: Kubernetes's CLI; used to update and check digi's states. 

In [1]:
from tutorial import (
    create,
    model_file,
    handler_file,
)
%elapsed_time

0:00:00


# Lamp digivice
----

This simple lamp digivice allows one to configure its power ("on" or "off") and brightness level.

## Define a schema

In [2]:
%%elapsed_time

schema = """
group: digi.dev
version: v1
kind: Lamp
control:
  power: string  
  # Add a 'brightness' attribute to the lamps's 
  # schema. The following data types are allowed: 
  # {number, integer, string, array,object}
  # YOUR CODE HERE
  brightness: number
"""

create(schema)

0:00:02


## Define a model

In [3]:
m = model_file("lamp")

In [4]:
%%elapsed_time
%%writefile $m

apiVersion: digi.dev/v1
kind: Lamp         
metadata:
  name: {{ .Values.name }}
spec:
  control:        
    power:
      intent: "on"
    brightness:
      intent: 0.8

Writing /Users/silv/go/src/digi.dev/tutorial/workdir/lamp/deploy/cr.yaml
0:00:05


## Implement a driver
----
**Goal:** Implement a "mock" driver that simply sets the lamp's status equal to its intent.

In [5]:
f = handler_file("lamp")

In [6]:
%%elapsed_time
%%writefile $f

from digi import on

@on.control("power")
def handle_power(sv):
    # sv (shorthand for "subview") gives acess
    # to the sub-tree of the lamp's model rooted
    # at the "power" attribute
    sv["status"] = sv["intent"]

@on.control("brightness")
def handle_brightness(sv):
    # set the status of brightness
    # YOUR CODE HERE
    sv["status"] = sv["intent"]

Overwriting /Users/silv/go/src/digi.dev/tutorial/workdir/lamp/driver/handler.py
0:00:07


## Build

In [7]:
!dq build lamp -q  # quiet
!dq image

IMAGE ID
lamp
motionsensor
room


## Run

In [8]:
!dq run lamp lamp-test
# !dq stop lamp lamp-test

lamp-test


## Read status

In [9]:
!kubectl get lamp.digi.dev lamp-test -oyaml | kubectl neat

apiVersion: digi.dev/v1
kind: Lamp
metadata:
  name: lamp-test
  namespace: default
spec:
  control:
    brightness:
      intent: 0.8
      status: 0.8
    power:
      intent: "on"
      status: "on"


## Update intent

One can modify the intent field of control attributes to update the desired states of the digivice. Here, let's update the desired brightness to 0.1 (previously 0.8).

In [10]:
m = model_file("lamp", new=False)

In [11]:
%%elapsed_time
%%writefile $m

apiVersion: digi.dev/v1
kind: Lamp         
metadata:
  name: lamp-test
spec:
  control:        
    power:
      intent: "on"
    brightness:
      intent: 0.1

Overwriting /Users/silv/go/src/digi.dev/tutorial/workdir/lamp/deploy/cr_run.yaml
0:02:19


In [12]:
!kubectl apply -f $m 2> /dev/null  
!kubectl get lamp lamp-test -oyaml | kubectl neat

lamp.digi.dev/lamp-test configured
apiVersion: digi.dev/v1
kind: Lamp
metadata:
  name: lamp-test
  namespace: default
spec:
  control:
    brightness:
      intent: 0.1
      status: 0.1
    power:
      intent: "on"
      status: "on"


In [13]:
# Alternatively, one can patch using a string (the previous method is preferred!), e.g.,
!kubectl patch lamp lamp-test -p '{"spec":{"control":{"power":{"intent":"on"}}}}' --type=merge

lamp.digi.dev/lamp-test patched (no change)


# HL abstraction: Room digivice
----

## Schema and model

In [14]:
%%elapsed_time

# define schema
schema = """
group: digi.dev
version: v1
kind: Room
control:
  brightness: number
mount:     
  digi.dev/v1/lamps: object
  # ... additional digis
"""

create(schema)

0:02:26


In [15]:
# specify model
m = model_file("room")

In [16]:
%%elapsed_time
%%writefile $m

apiVersion: digi.dev/v1
kind: Room        
metadata:
  name: {{ .Values.name }}
spec:
  control:        
    brightness:
      intent: 0.8

Writing /Users/silv/go/src/digi.dev/tutorial/workdir/room/deploy/cr.yaml
0:02:29


## Implement a driver
----
**Goal:** The room digivice aggregates the brightness of all lamps mounted to it. This means:
* The `control.brightness.status` should be equal to the sum of the corresponding `status` attribute of the lamps that are currently active (i.e., `power.intent == "on"`)
* The value of `control.brightness.intent` should be divided across all lamps

For example, given a room that has 10 lamps, if the room's `control.brightness.intent` is set to 1.0 and 5 of the lamps have `control.power.status == "on"`, then each of these active lamp's `control.brightness.intent` should be set to 1.0/5 = `0.2`.

In [17]:
f = handler_file("room")

In [18]:
%%elapsed_time
%%writefile $f

from digi import on
from digi.view import TypeView, DotView

@on.mount
@on.control
def handle_brightness(proc_view):    
    with TypeView(proc_view) as tv, DotView(tv) as dv:  
        # reference to room's brightness attribute;
        # it contains two sub-fields .intent and .status
        room_brightness = dv.root.control.brightness        
        room_brightness.status = 0
        
        if "lamps" not in dv:
            return

        active_lamps = [l for _, l in dv.lamps.items() 
                        if l.control.power.status == "on"]
        for lamp in active_lamps:
            room_brightness.status += lamp.control.brightness.status 
            
            # update lamp's intent
            # YOUR CODE HERE
            lamp.control.brightness.intent = room_brightness.intent / len(active_lamps) 

Overwriting /Users/silv/go/src/digi.dev/tutorial/workdir/room/driver/handler.py
0:02:32


In [19]:
# build and run
!dq build room -q
!dq run room room-test

room-test


## Debug
----

To add additional debug info:

```python
from digi import logger
@control
def h(proc_view):
    logger.info("...")
```

In [20]:
!kubectl get rooms room-test -oyaml | kubectl neat

apiVersion: digi.dev/v1
kind: Room
metadata:
  name: room-test
  namespace: default
spec:
  control:
    brightness:
      intent: 0.8


In [21]:
!dq log room-test

[2021-04-25 23:10:14,300] digi                 [INFO    ] Started an operator
[2021-04-25 23:10:14,300] digi.mount           [INFO    ] Started the mounter
[2021-04-25 23:10:14,300] digi.mount           [INFO    ] Started the mounter
[2021-04-25 23:10:14,304] kopf.reactor.running [WARNING ] OS signals are ignored: running not in the main thread.
[2021-04-25 23:10:14,304] digi                 [INFO    ] Started an operator
[2021-04-25 23:10:14,304] digi                 [INFO    ] Started an operator
[2021-04-25 23:10:14,305] kopf.reactor.running [WARNING ] OS signals are ignored: running not in the main thread.
[2021-04-25 23:10:14,616] digi.main            [INFO    ] Done reconciliation
[2021-04-25 23:10:14,616] digi.main            [INFO    ] Done reconciliation
[2021-04-25 23:10:14,728] digi.main            [INFO    ] Skipping gen 2
[2021-04-25 23:10:14,728] digi.main            [INFO    ] Skipping gen 2


## Mount lamps

In [22]:
!dq mount lamp-test room-test

In [23]:
!kubectl get rooms room-test -oyaml | kubectl neat

apiVersion: digi.dev/v1
kind: Room
metadata:
  name: room-test
  namespace: default
spec:
  control:
    brightness:
      intent: 0.8
      status: 0.8
  mount:
    digi.dev/v1/lamps:
      default/lamp-test:
        mode: hide
        spec:
          control:
            brightness:
              intent: 0.8
              status: 0.8
            power:
              intent: "on"
              status: "on"
        status: active


In [24]:
# start a new lamp-test-2
!dq run lamp lamp-test-2

lamp-test-2


In [25]:
!dq mount lamp-test-2 room-test

In [27]:
!kubectl get rooms room-test -oyaml | kubectl neat

apiVersion: digi.dev/v1
kind: Room
metadata:
  name: room-test
  namespace: default
spec:
  control:
    brightness:
      intent: 0.8
      status: 0.8
  mount:
    digi.dev/v1/lamps:
      default/lamp-test:
        mode: hide
        spec:
          control:
            brightness:
              intent: 0.4
              status: 0.4
            power:
              intent: "on"
              status: "on"
        status: active
      default/lamp-test-2:
        mode: hide
        spec:
          control:
            brightness:
              intent: 0.4
              status: 0.4
            power:
              intent: "on"
              status: "on"
        status: active


Or [watch the changes](http://localhost:8881/notebooks/display.ipynb).

## Playing with room brightness

In [28]:
!kubectl patch room room-test -p '{"spec":{"control":{"brightness":{"intent":1}}}}' --type=merge

room.digi.dev/room-test patched


In [30]:
!kubectl get rooms room-test -oyaml | kubectl neat

apiVersion: digi.dev/v1
kind: Room
metadata:
  name: room-test
  namespace: default
spec:
  control:
    brightness:
      intent: 1
      status: 1
  mount:
    digi.dev/v1/lamps:
      default/lamp-test:
        mode: hide
        spec:
          control:
            brightness:
              intent: 0.5
              status: 0.5
            power:
              intent: "on"
              status: "on"
        status: active
      default/lamp-test-2:
        mode: hide
        spec:
          control:
            brightness:
              intent: 0.5
              status: 0.5
            power:
              intent: "on"
              status: "on"
        status: active


## Connect to physical lamps

...

# Activate the Room with motion
----
**Goal:** Allow the room to detect motion and adjust the brightness level accordingly. 

We will use a mock motion sensor (`mock.digi.dev/v1/motionsensors`) which "pretends to detect" motion event randomly over time. 
* When a motion is detected, the motion sensor updates its `obs.last_triggered_time` to the current time
* The motion sensor has a `sensitivity` control attribute that decides how often the motion is generated

## Pull the MotionSensor

In [33]:
!dq pull motionsensor
!dq run motionsensor motion-test

motionsensor
motion-test


In [34]:
!kubectl get motionsensor motion-test -oyaml | kubectl neat

apiVersion: mock.digi.dev/v1
kind: MotionSensor
metadata:
  name: motion-test
  namespace: default
spec:
  control:
    sensitivity:
      intent: 1
      status: 1
  obs:
    battery_level: 100%


In [35]:
# update the sensor's sensitivity (default to 1) 
# s.t. it generates events 10 times more often
!kubectl patch motionsensor motion-test -p '{"spec":{"control":{"sensitivity":{"intent":10}}}}' --type=merge

motionsensor.mock.digi.dev/motion-test patched


In [38]:
!kubectl get motionsensor motion-test -oyaml | kubectl neat

apiVersion: mock.digi.dev/v1
kind: MotionSensor
metadata:
  name: motion-test
  namespace: default
spec:
  control:
    sensitivity:
      intent: 10
      status: 10
  obs:
    battery_level: 100%
    last_triggered_time: 1.619392494784037e+09


## Modify Room

In [39]:
schema = """
group: digi.dev
version: v1
kind: Room
control:
  brightness: number
mount:     
  digi.dev/v1/lamps: object
  # allow Room to mount the motion sensor
  mock.digi.dev/v1/motionsensors: object
reflex: object
"""

create(schema, new=False)

In [41]:
# build and run
!dq build room -q
!dq run room room-test

room-test


## Mount a motionsensor

In [42]:
!dq mount motion-test room-test

In [43]:
!dq mount lamp-test room-test

In [45]:
!dq mount lamp-test-2 room-test

In [46]:
!kubectl get room room-test -oyaml | kubectl neat

apiVersion: digi.dev/v1
kind: Room
metadata:
  name: room-test
  namespace: default
spec:
  control:
    brightness:
      intent: 0.8
      status: 0.8
  mount:
    digi.dev/v1/lamps:
      default/lamp-test:
        mode: hide
        spec:
          control:
            brightness:
              intent: 0.4
              status: 0.4
            power:
              intent: "on"
              status: "on"
        status: active
      default/lamp-test-2:
        mode: hide
        spec:
          control:
            brightness:
              intent: 0.4
              status: 0.4
            power:
              intent: "on"
              status: "on"
        status: active
    mock.digi.dev/v1/motionsensors:
      default/motion-test:
        mode: hide
        spec:
          control:
            sensitivity:
              intent: 10
              status: 10
          obs:
            battery_level: 100%
            last_triggered_time: 1.6193925240519307e+09
        status: active

## Add Reflex
----

A reflex defines a policy/logic on the model itself. It allows users to augment or modify the behaviors of a digi at runtime without restarting or rebuilding it. 

A reflex has three attribtes: `policy` defines the logic in a processor specific language (e.g., jq - a popular JSON processor); `priority` defines which priority level this policy will run at (handlers in the driver have default priority of 0; a negative priority means the reflex is disabled); the `processor` defines which processor will be used. 

Each logic has a name which is its root attribute. One can use the reflex to reconfigure an existing handler's logic and priority too, by simply use that handler's name as the reflex's name.

In the following example, we are going to add a reflex that updates room's brightness when there is motion.

In [47]:
# specify model
m = model_file("room", new=False)

In [48]:
%%elapsed_time
%%writefile $m

apiVersion: digi.dev/v1
kind: Room
metadata:
  name: room-test
spec:
  control:
    brightness:
      intent: 0.8
  reflex:
    motion-mode:  
      # if the motion is detected recently (less than 10 minutes),
      # set the room's brightness to 1
      # YOUR CODE HERE 
      # Hint: fill in the missing (???) attributes
      policy: >- # 
            if $time - ."motion-test".obs.last_triggered_time <= 600 
            then .root.control.brightness.intent = 1 else . end  
      priority: 0
      processor: jq

Overwriting /Users/silv/go/src/digi.dev/tutorial/workdir/room/deploy/cr_run.yaml
0:07:59


In [49]:
!kubectl apply -f $m 2> /dev/null  

room.digi.dev/room-test configured


In [51]:
!kubectl get room room-test -oyaml | kubectl neat

apiVersion: digi.dev/v1
kind: Room
metadata:
  name: room-test
  namespace: default
spec:
  control:
    brightness:
      intent: 1
      status: 1
  mount:
    digi.dev/v1/lamps:
      default/lamp-test:
        mode: hide
        spec:
          control:
            brightness:
              intent: 0.5
              status: 0.5
            power:
              intent: "on"
              status: "on"
        status: active
      default/lamp-test-2:
        mode: hide
        spec:
          control:
            brightness:
              intent: 0.5
              status: 0.5
            power:
              intent: "on"
              status: "on"
        status: active
    mock.digi.dev/v1/motionsensors:
      default/motion-test:
        mode: hide
        spec:
          control:
            sensitivity:
              intent: 10
              status: 10
          obs:
            battery_level: 100%
            last_triggered_time: 1.6193925461424692e+09
        status: active
  r

## Connect to real motion detectors

...

# Post tutorial question
> Which part did you find most difficult?

> Any suggestions on the programming/tools? 

# Home (Bonus)

## Pull the base image

In [ ]:
!dq pull home

## Modify the Home driver
----
**Goal:** Home have a "mode" control attribute that allows one to tune it to predefined modes. Each mode decides the brightness of the Rooms that mounted to the Home.

In [ ]:
f = handler_file("home")

In [ ]:
%%elapsed_time
%%writefile $f

# TBD
import digi
import digi.on as on

# validation
@on.attr
def h():
    ...

# intent back-prop
@on.mount
def h():
    ...

# status
@on.mount
def h():
    ...

# intent
@on.mount
@on.control
def h():
    ...

In [ ]:
# build and run
!dq build home -q
!dq run home home-test

## Mount rooms

In [ ]:
!dq mount room-test home-test

In [ ]:
# set the modes
!kubectl patch home home-test -p '{"spec":{"control":{"mode":{"intent":YOURMODE}}}}' --type=merge